## Step 1 Install Dependencis 

In [0]:
!pip install -q tensorflow-gpu==2.0.0-beta1

## Step 2 Imports

In [2]:
import time
import numpy as np
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

## Step 3 Dataset Preprocessing

### Loading the MNIST Dataset

In [0]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

### Image Normalization

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [5]:
X_train.shape

(60000, 28, 28)

### Dataset reshaping

In [0]:
X_train = X_train.reshape(-1,28*28)
X_test = X_test.reshape(-1,28*28)

In [7]:
X_train.shape

(60000, 784)

## Step 4 Distributed Training

### Defining a non distributed model

In [0]:
model_normal = tf.keras.models.Sequential()

### Building the non distributed model

In [0]:
model_normal.add(tf.keras.layers.Dense(units=128, activation = 'relu', input_shape=(784,)))

In [0]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [0]:
#output layer. must have same number of units as classes in dataset
model_normal.add(tf.keras.layers.Dense(units = 10, activation = 'softmax'))

### Compile the model

In [12]:
model_normal.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
model_normal.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Defining the Distributed Strategy

In [0]:
distribute = tf.distribute.MirroredStrategy()

### Defining a Distributed Model

In [0]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate = 0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation = 'softmax'))
  model_distributed.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])

##Speed comparison for proof of concept. 

In [15]:
start_time = time.time()
model_distributed.fit(X_train,y_train, epochs = 10, batch_size=25)
print(f"Distributed training took: {time.time()-start_time}")

Train on 2400 steps
Epoch 1/10
2400/2400 [==============================] - 14s 6ms/step - loss: 0.2785 - sparse_categorical_accuracy: 0.9191
Epoch 2/10
2400/2400 [==============================] - 10s 4ms/step - loss: 0.1388 - sparse_categorical_accuracy: 0.9584
Epoch 3/10
2400/2400 [==============================] - 11s 4ms/step - loss: 0.1059 - sparse_categorical_accuracy: 0.9672
Epoch 4/10
2400/2400 [==============================] - 11s 5ms/step - loss: 0.0871 - sparse_categorical_accuracy: 0.9729
Epoch 5/10
2400/2400 [==============================] - 11s 4ms/step - loss: 0.0746 - sparse_categorical_accuracy: 0.9758
Epoch 6/10
2400/2400 [==============================] - 11s 4ms/step - loss: 0.0642 - sparse_categorical_accuracy: 0.9797
Epoch 7/10
2400/2400 [==============================] - 10s 4ms/step - loss: 0.0578 - sparse_categorical_accuracy: 0.9807
Epoch 8/10
2400/2400 [==============================] - 11s 4ms/step - loss: 0.0527 - sparse_categorical_accuracy: 0.9833
Epoc

In [16]:
start_time = time.time()
model_normal.fit(X_train,y_train, epochs = 10, batch_size=25)
print(f"Nomal training took: {time.time()-start_time}")

W0813 17:28:11.423160 140708692191104 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.2807 - sparse_categorical_accuracy: 0.9176
Epoch 2/10
60000/60000 [==============================] - 13s 222us/sample - loss: 0.1370 - sparse_categorical_accuracy: 0.9585
Epoch 3/10
60000/60000 [==============================] - 13s 209us/sample - loss: 0.1027 - sparse_categorical_accuracy: 0.9687
Epoch 4/10
60000/60000 [==============================] - 12s 204us/sample - loss: 0.0851 - sparse_categorical_accuracy: 0.9733
Epoch 5/10
60000/60000 [==============================] - 12s 206us/sample - loss: 0.0740 - sparse_categorical_accuracy: 0.9768
Epoch 6/10
60000/60000 [==============================] - 12s 204us/sample - loss: 0.0636 - sparse_categorical_accuracy: 0.9793
Epoch 7/10
60000/60000 [==============================] - 12s 200us/sample - loss: 0.0577 - sparse_categorical_accuracy: 0.9811
Epoch 8/10
60000/60000 [==============================] - 12s 203us/sample - loss